In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [2]:
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]

'/home/mori/.gcloud/agent-engine-test-461308-a2228f8932d9.json'

## Cloud SQL Proxy を通してユーザ名とパスワードで接続する


In [ ]:
# !nohup cloud-sql-proxy --address 127.0.0.1 --port 5432 agent-engine-test-461308:us-central1:agent-engine-test &

In [4]:
project_id = "agent-engine-test-461308"
region = "us-central1"
instance_name = "agent-engine-test"
database_name = "testdb"
db_user = "postgres"
db_password = "postgres"

In [ ]:
import psycopg2


try:
    conn = psycopg2.connect(
        host="127.0.0.1",
        port="5432",
        user=db_user,
        password=db_password,
        database=database_name,
    )
    cur = conn.cursor()
    cur.execute("SELECT * FROM mytable;")
    rows = cur.fetchall()

    for row in rows:
        print(row)

    cur.close()
    conn.close()
except Exception as e:
    print(f"Connection error: {e}")

(1, 'Alice', 30)
(2, 'Bob', 25)
(3, 'Charlie', 35)


## Cloud SQL Python Connector を通して接続する

- サービスアカウントのロールに `Cloud SQL クライアント` を追加する


In [5]:
project_id = "agent-engine-test-461308"
region = "us-central1"
instance_name = "agent-engine-test"
database_name = "testdb"
db_user = "postgres"
db_password = "postgres"

In [10]:
from google.cloud.sql.connector import Connector

connector = Connector()


def getconn():
    conn = connector.connect(
        f"{project_id}:{region}:{instance_name}",
        "pg8000",
        user=db_user,
        password=db_password,
        db=database_name,
    )
    return conn


try:
    with getconn() as conn:
        cur = conn.cursor()
        cur.execute("SELECT * FROM mytable;")
        rows = cur.fetchall()
        for row in rows:
            print(row)
        cur.close()
except Exception as e:
    print(e)

[1, 'Alice', 30]
[2, 'Bob', 25]
[3, 'Charlie', 35]
